## Бражникова Ольга

## Предварительные требования для реализации

1) Развернуть бесплатный Qdrant-кластер в облаке через Qdrant Cloud и получить URL подключения и API-ключ — они требуются для инициализации клиента Qdrant.
2) Создать Telegram-бота через @BotFather, получить токен и указать его в конфигурации проекта — он используется для подключения агента к Telegram.
3) Получить API-ключ от провайдера LLM (например, Deepseek, OpenAI, или другого), так как агент использует языковую модель для генерации ответов — ключ должен быть сохранён в конфигурационном файле.
4) Склонировать проект и установить менеджер зависимостей uv: выполнить pip install uv в окружении.
5) Установить зависимости проекта: запустить uv sync, чтобы восстановить все необходимые Python-библиотеки из pyproject.toml / uv.lock.
6) Загрузить и проиндексировать данные в Qdrant: запустить все ячейки в данном ipynb
7) Запустить основной скрипт агента: выполнить uv run python -m src.rag_bot.main, который инициализирует AutoGen-агентов, подключает их к Telegram и настраивает RAG-логику поверх Qdrant.

## Загрузка данных в Qdrant


Скачиваем и векторизуем информацию по винам. Загружает список файлов из папки data/wines.txt репозитория GitHub через API, затем поочерёдно скачиваем содержимое каждого файла и сохраняем тексты в список wines.




In [ ]:
import requests
wine_folder_info = requests.get("https://api.github.com/repos/yandex-datasphere/advanced-assistant/contents/data/wines.txt").json()
wines = []
for file_info in wine_folder_info:
  download_url = file_info["download_url"]
  r = requests.get(download_url)
  wine_text = r.content.decode()
  wines.append(wine_text)
print(wines)

['## Альбариньо \n\nАльбариньо\nАльбариньо (исп. *Albariño*) — сорт белого винного винограда родом из Галисии на Северо-Атлантическом побережье Испании. Название происходит от от *albo* или *albus*, что означает белый. Сорт отлично растет во влажном климате и в непосредственной близости к морю, но, несмотря на толстую кожицу ягод и выносливость лозы, чувствителен к плесени и гнили. Поэтому виноделам необходимо заботиться о дополнительном дренаже почв. Пальму первенства по выращиванию альбариньо делят Испания и Португалия.\nДо недавнего времени считалось, что сорт альбариньо появился в долине Рейна, и в Галисию его завезли монахи-цистерцианцы из французской коммуны Клюни в XII веке. Однако ученые отдела виноградарства Центра биологических исследований в Понтеведра выяснили: родина сорта — Галисия, и вино альбариньо производилось здесь еще со времен Римской империи.\nИз альбариньо производят легкие (до 12%) белые вина с высокой кислотностью и свежим ароматом с нотами персика, нектарина, 

Устанавливаем библиотеку FlagEmbedding, необходимую для работы с эмбеддингами.

In [ ]:
!pip install FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.1 MB/s eta 0:00:00
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.3.5-py3-none-any.whl size=233746 sha256=bf67df38f89e040e2992e9859a6fdefffe328f956633fc9eb9e48e86e80718a6
  Stored in directory: /root/.cache/pip/wheels/b2/1f/f6/78f862bb80cb959cc9960b7c4e2d1f702b1bc0e79d19b5f124
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=af7b5cc8085a32dbd2

Загружаем предобученную модель BAAI/bge-m3, затем генерируем плотные (dense) векторные представления (эмбеддинги) для всех текстов из списка wines, обрабатывая их пакетами по 15 и сохраняем результат в переменную output.

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3')


# Получаем dense-эмбеддинги
output = model.encode(
    wines,
    batch_size=15,
    max_length=8192,
    return_dense=True,
    return_sparse=False,
    return_colbert_vecs=False
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

...

onnx/tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

pre tokenize: 100%|██████████| 2/2 [00:00<00:00, 32.11it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 2/2 [00:00<00:00,  2.55it/s]


In [ ]:
wines_dense = output['dense_vecs']

Аналогичным образом скачиваем и векторизуем информацию по регионам

In [ ]:
import requests
regions_folder_info = requests.get("https://api.github.com/repos/yandex-datasphere/advanced-assistant/contents/data/regions.txt").json()
regions = []
for file_info in regions_folder_info:
  download_url = file_info["download_url"]
  r = requests.get(download_url)
  regions_text = r.content.decode()
  regions.append(regions_text)
print(regions)

["## Абруццо \n\nАбруццо\nАбруццо — регион на Адриатическом побережье Италии, расположенный практически посередине сапожка. Соседствует с Марке, Лацио и Молизе. На территории немало заповедников и национальных парков, а большинство городов и поселений здесь основаны в Средние века или в период Возрождения. Административный центр — город Л'Акуила. В 2009 году он стал всемирно известным из-за масштабного и разрушительного землетрясения.\nВ регионе есть лишь одна зона производства вин наивысшего качества DOCG — Монтепульчано д’Абруццо Коллине Терамане. И множество зон вин среднего качества с контролируемым происхождением. \nБольшинство виноградников расположены на холмах одной из самых известных винодельческих провинций — Кьети. Сейчас вина из Абруццо поставляются не только в страны ЕС, но и в Великобританию, США и Канаду.\nИстинные звезды Абруццо — красный сорт монтепульчано и белый треббьяно. Именно они, как правило, входят в состав вин, производимых в регионе. \nИз других характерных и

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3')


# Получаем dense-эмбеддинги
output = model.encode(
    regions,
    batch_size=15,
    max_length=8192,
    return_dense=True,
    return_sparse=False,
    return_colbert_vecs=False
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 7/7 [00:00<00:00, 54.45it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 7/7 [00:02<00:00,  2.99it/s]


In [ ]:
regions_dense = output['dense_vecs']

Скачиваем и векторизуем информацию по подходящим к вину блюдам. Получаем содержимое файла food_wine_table.md из GitHub через API и сохраняем его как строку food_text.

In [ ]:
import requests
food_info = requests.get("https://api.github.com/repos/yandex-datasphere/advanced-assistant/contents/data/food_wine_table.md").json()
download_url = food_info["download_url"]
r = requests.get(download_url)
food_text = r.content.decode()
print(food_text)

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.
Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).
Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.
Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Вермент

Парсим markdown-таблицу: пропускаем заголовки, разбиваем строки по символу |, извлекаем пары «блюдо — вино» и сохраняем их в виде списка словарей records, затем формируем из них текстовые описания вида «Блюдо: ... Подходит вино: ...» и складываем в список dishes.

In [ ]:
import re

lines = food_text.strip().split('\n')[2:]
records = []
for line in lines:
    if '|' in line:
        parts = [part.strip() for part in line.split('|')]
        if len(parts) == 2:
            dish, wine = parts
            records.append({"dish": dish, "wine": wine})
print(records)
dishes = []
for rec in records:
    dish = f"Блюдо: {rec['dish']}. Подходит вино: {rec['wine']}."
    dishes.append(dish)
print(dishes)

[{'dish': 'Баклажаны, запеченые с сыром', 'wine': 'Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.'}, {'dish': 'Баранина деликатесная (филе или каре ягненка)', 'wine': 'Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).'}, {'dish': 'Баранина пикантная: жареная, гриль, тушеная — со специями', 'wine': 'Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.'}, {'dish': 'Бефстроганов', 'wine': 'Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Верментино. Не о

Генерируем dense эмбеддинги

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

# Векторизуем объединённые документы
output = model.encode(
    dishes,
    batch_size=16,
    max_length=512,
    return_dense=True,
    return_sparse=False,
    return_colbert_vecs=False
)

food_dense = output['dense_vecs']


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 10/10 [00:00<00:00, 151.01it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 10/10 [00:00<00:00, 14.17it/s]


Прогрузим в Qdrant информацию по винам, регионам и блюдам. Устанавливаем библиотеку qdrant-client для взаимодействия с векторной базой данных Qdrant.


In [ ]:
!pip install qdrant-client

Создаём клиент Qdrant, подключаясь к удалённому кластеру по указанному URL и API-ключу, и выводим список существующих коллекций.


In [ ]:
from qdrant_client import QdrantClient


qdrant_client = QdrantClient(
    url="ваш url для подключения",
    api_key="ваш апи ключ",
)

print(qdrant_client.get_collections())


collections=[]


Проверяем наличие коллекции с именем wines: если её нет — создаёт новую с конфигурацией для плотных векторов размерностью 1024 и метрикой косинусного расстояния, а также поддержкой разреженных BM25-векторов.

In [ ]:
from qdrant_client.models import VectorParams, Distance, SparseVectorParams

collection_name = "wines"

if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
      collection_name="wines",
      vectors_config={
          "dense": VectorParams(size=1024, distance=Distance.COSINE)
      },
      sparse_vectors_config={
          "bm25": SparseVectorParams()
      }
)


Прогружаем информацию по винам. Формирует список точек (PointStruct) на основе текстов вин: каждая точка содержит уникальный UUID, dense вектор, BM25-представление текста и метадату. Затем все точки загружаем (upsert) в коллекцию wines.

In [ ]:
from qdrant_client.models import  PointStruct, Document
import uuid
points = []
for i, text in enumerate(wines):
    points.append(
          PointStruct(
            id=str(uuid.uuid4()),
            vector={
                "dense": wines_dense[i].tolist(),
                "bm25": Document(
                    text=text,
                    model="Qdrant/bm25"
                )
            },
            payload={
                "text": text,
                "label":"wines"
            }
        )
    )
qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

Аналогично прогружаем информацию по регионам.

In [ ]:
from qdrant_client.models import  PointStruct, Document
import uuid
points = []
for i, text in enumerate(regions):
    points.append(
          PointStruct(
            id=str(uuid.uuid4()),
            vector={
                "dense": regions_dense[i].tolist(),
                "bm25": Document(
                    text=text,
                    model="Qdrant/bm25"
                )
            },
            payload={
                "text": text,
                "label":"regions"
            }
        )
    )
qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

Аналогично прогружаем информацию по блюдам

In [ ]:
from qdrant_client.models import  PointStruct, Document
import uuid
points = []
for i, text in enumerate(dishes):
    points.append(
          PointStruct(
            id=str(uuid.uuid4()),
            vector={
                "dense":  food_dense[i].tolist(),
                "bm25": Document(
                    text=text,
                    model="Qdrant/bm25"
                )
            },
            payload={
                "text": text,
                "label":"food_wine"
            }
        )
    )
qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

Проверим работоспособность поиска

In [ ]:
from qdrant_client.models import Fusion, FusionQuery, Prefetch

query_text = "Карменер Карменер"

dense_query = model.encode([query_text])['dense_vecs'][0]

hits = qdrant_client.query_points(
    collection_name="wines",
    prefetch=[
        Prefetch(
            query=dense_query.tolist(),
            using="dense"
        ),
        Prefetch(
            query=Document(text=query_text, model="qdrant/bm25"),
            using="bm25"
        )
    ],
    query=FusionQuery(fusion=Fusion.RRF),
    limit=3
)

for hit in hits.points:
    print(hit.payload["text"])

## Карменер 

Карменер
Карменер (фр. Сarmenere) — сорт красного винного винограда из семейства каберне и родом из Бордо. Название образовано от французского слова carmin — малиновый. Виноград получил его за оттенок листьев осенью. В конце XIX века филлоксера уничтожила карменер по всему Бордо. Но виноград обрел новый дом — в Чили, где сегодня считается флагманским сортом. По масштабам посадок с Чили конкурирует только Китай. Также виноград выращивают в Италии, Новой Зеландии и Австралии.
Согласно записям Плиния Старшего, карменер появился на территории современной Испании. Уже позднее римляне высадили виноград в регионе Медок, где он стал одним из шести разрешенных сортов для купажей Бордо вплоть до нашествия филлоксеры в 1867 году.
Исследования ДНК показали, что карменер имеет много общего с сортами каберне совиньон и каберне фран.
Из карменера получают красные сухие вина средней полноты с гладким танинным профилем и умеренной кислотностью. Крепость варьируется в диапазоне от 13 до 15

In [ ]:
from qdrant_client.models import Fusion, FusionQuery, Prefetch

query_text = "Азорские острова"

dense_query = model.encode([query_text])['dense_vecs'][0]

hits = qdrant_client.query_points(
    collection_name="wines",
    prefetch=[
        Prefetch(
            query=dense_query.tolist(),
            using="dense"
        ),
        Prefetch(
            query=Document(text=query_text, model="qdrant/bm25"),
            using="bm25"
        )
    ],
    query=FusionQuery(fusion=Fusion.RRF),
    limit=1
)

for hit in hits.points:
    print(hit.payload["text"])

## Азорские острова 

Азорские острова
Азорские острова — самобытный винодельческий регион Португалии, расположенный на архипелаге посреди Атлантического океана. Уникальность технологий виноградарства, как и качество и вкус вина в регионе, обуславливают климат и своеобразный почвенный состав. Регион разделен на три субрегиона (апелласьона): самый крупный Пико (остров Пику), Бискоитос (остров Терсейра) и Грасиоза (остров Грасиоза).
Острова Азорского архипелага имеют вулканическое происхождение, что обуславливает своеобразный состав почв. Чтобы высадить первые виноградники на острове Пику, виноградари сначала расчищали землю от вулканических камней, стены из которых стали своеобразной защитой для плантаций. 
В составе почв встречаются следы базальтовых пород, окаменелой лавы, вулканического песка. Большое влияние на виноградарство оказывает Атлантический океан. 
Разрешенными признаны 33 сорта винограда, из которых 15 составляют красные сорта и 18 — белые. Часть из них — международные и п

In [ ]:
from qdrant_client.models import Fusion, FusionQuery, Prefetch

query_text = "Какое вино подходит к баранине на гриле со специями?"

dense_query = model.encode([query_text])['dense_vecs'][0]

hits = qdrant_client.query_points(
    collection_name="wines",
    prefetch=[
        Prefetch(
            query=dense_query.tolist(),
            using="dense"
        ),
        Prefetch(
            query=Document(text=query_text, model="qdrant/bm25"),
            using="bm25"
        )
    ],
    query=FusionQuery(fusion=Fusion.RRF),
    limit=3
)

for hit in hits.points:
    print(hit.payload.get("text","Без текста"))

Блюдо: Баранина пикантная: жареная, гриль, тушеная — со специями. Подходит вино: Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России..
## Рейнхессен 

Рейнхессен
Рейнхессен (нем. Rheinhessen) — самый крупный винодельческий регион Германии, который находится на левом берегу реки Рейна. Из-за особенностей рельефа его называют землей тысячи холмов. Рейнхессен известен как поставщик простых белых вин с легким вкусом.
Рейнхессен — солнечный регион с небольшим количеством осадков. Уникальный состав местных почв придает винам сланцевые ноты. Рейнхессен состоит из трех винодельческих районов, которые носят названия Ротер Ханг, Накенхайм, Ниерштайн.
Последний расположен рядом с Рейном, благодаря чему его почвы на

Добавим информацию по цене вин. Загружаем Excel-файл с данными о винах, читаем его в DataFrame и с помощью модели BAAI/bge-m3 генерируем плотные эмбеддинги для названий вин, сохраняя их в столбец vector.

In [ ]:
import pandas as pd
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3')
df_ru = pd.read_excel("https://github.com/yandex-datasphere/advanced-assistant/raw/main/data/wine-price-ru.xlsx")
# Генерация векторов из названий
df_ru["vector"] = df_ru["Name"].apply(lambda x: model.encode([str(x)],return_dense=True)["dense_vecs"][0].tolist())

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
df_ru

,Id,Name,Country,Price,WHPrice,etc,Acidity,Color,Volume,vector
0,56885,САССИКАЙЯ КР СХ,IT,27799.000,19459.3000,NaN,Сухое,Красное,0.750,"[-0.00687408447265625, 0.0281219482421875, -0...."
1,666560,СИЕПИ МАЗЕЙ КР СХ,IT,15999.000,11199.3000,NaN,Сухое,Красное,0.750,"[0.048858642578125, 0.02276611328125, -0.03955..."
2,533769,ПАЛАФРЕНО КР СХ,IT,14999.004,10499.3028,NaN,Сухое,Красное,0.750,"[0.029296875, 0.06024169921875, -0.02957153320..."
3,93733,АНТ ТИНЬЯНЕЛЛО КР СХ,IT,14499.012,10149.3084,NaN,Сухое,Красное,0.750,"[-0.022369384765625, 0.08636474609375, -0.0116..."
4,644863,ШАТО МОНРОЗ КР СХ,FR,12999.000,9099.3000,от промо цены,Сухое,Красное,0.750,"[0.0159454345703125, 0.0309295654296875, -0.01..."
...,...,...,...,...,...,...,...,...,...,...
747,61418,КАГОР ТАМ КР СЛ,RU,179.004,125.3028,от промо цены,Сладкое,Красное,0.700,"[-0.0243377685546875, 0.01438140869140625, -0...."
748,615581,ДЖАСТ МЕРЛО КР СХ,FR,149.004,104.3028,NaN,Сухое,Красное,0.187,"[-0.0156402587890625, 0.017791748046875, -0.03..."
749,615582,ДЖАСТ КБСВ КР СХ,FR,149.004,104.3028,NaN,Сухое,Красное,0.187,"[-0.0016317367553710938, 0.0302581787109375, -..."
750,83302,АДАГУМ КБСВ КР СХ,RU,119.004,83.3028,NaN,Сухое,Красное,0.187,"[0.004974365234375, 0.019744873046875, -0.0252..."


Создаём точки (PointStruct) для загрузки в Qdrant. Все точки добавляются в коллекцию wines через upsert.

In [ ]:
from qdrant_client.models import  PointStruct, Document
import uuid
points = []

for idx, row in df_ru.iterrows():
    payload = {
        "Name": str(row["Name"]),
        "Color": str(row["Color"]),
        "Acidity": str(row["Acidity"]),
        "Country": str(row["Country"]),
        "Price": float(row["Price"]),
        "Volume": float(row["Volume"]),
    }
    points.append(PointStruct(
        id=str(uuid.uuid4()),
        vector={
                "dense": row["vector"],
                "bm25": Document(
                    text=row["Name"],
                    model="Qdrant/bm25"
                )
            },
        payload=payload
    ))
collection_name = "wines"
qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)


UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

Создаём индексы в Qdrant для полей полезной нагрузки (payload), чтобы ускорить фильтрацию: текстовые поля (Country, Name) индексируются как TEXT, категориальные (Color, Acidity) — как KEYWORD, числовые (Price) — как FLOAT.

In [ ]:
from qdrant_client.http.models import PayloadSchemaType


qdrant_client.create_payload_index(
    collection_name="wines",
    field_name="Country",
    field_schema=PayloadSchemaType.TEXT
)

qdrant_client.create_payload_index(
    collection_name="wines",
    field_name="Color",
    field_schema=PayloadSchemaType.KEYWORD
)


qdrant_client.create_payload_index(
    collection_name="wines",
    field_name="Name",
    field_schema=PayloadSchemaType.TEXT
)


qdrant_client.create_payload_index(
    collection_name="wines",
    field_name="Price",
    field_schema=PayloadSchemaType.FLOAT
)

qdrant_client.create_payload_index(
    collection_name="wines",
    field_name="Acidity",
    field_schema=PayloadSchemaType.KEYWORD
)

UpdateResult(operation_id=14, status=<UpdateStatus.COMPLETED: 'completed'>)

Для проверки выполним запрос к коллекции с фильтром по цене (вина дороже 100)

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter, FieldCondition, Range


filter_condition = Filter(
    must=[
        FieldCondition(
            key="Price",
            range=Range(gt=100)
        )
    ]
)


hits, _ = qdrant_client.scroll(
    collection_name="wines",
    scroll_filter=filter_condition,
    limit=10,
    with_payload=True,
    with_vectors=False
)


for point in hits:
    print(point.payload)

{'Name': ' ШАТО ЖАН ДЕ БЕЛЬ ЭР КР СХ', 'Color': 'Красное', 'Acidity': 'Сухое', 'Country': 'FR', 'Price': 648.996, 'Volume': 0.75}
{'Name': ' ТАНТАСЬОН ЛИРАК КРСХ', 'Color': 'Красное', 'Acidity': 'Сухое', 'Country': 'FR', 'Price': 1099.0, 'Volume': 0.75}
{'Name': ' КАМПО ВЬЕХ ТЕМП КРСХ', 'Color': 'Красное', 'Acidity': 'Сухое', 'Country': 'ES', 'Price': 259.0, 'Volume': 0.187}
{'Name': ' КАНТИ МЕРЛО КР ПСХ', 'Color': 'Красное', 'Acidity': 'Полусухое', 'Country': 'IT', 'Price': 549.0, 'Volume': 0.75}
{'Name': ' ПЕНЬЯЛОЛЕН КБСВ КР СХ', 'Color': 'Красное', 'Acidity': 'Сухое', 'Country': 'CL', 'Price': 1149.0, 'Volume': 0.75}
{'Name': ' НИНЬО ДЕ ЛАС КРСХ', 'Color': 'Красное', 'Acidity': 'Сухое', 'Country': 'ES', 'Price': 629.0, 'Volume': 0.75}
{'Name': ' КАПАННЕЛ СОЛАРЕ КР СХ', 'Color': 'Красное', 'Acidity': 'Сухое', 'Country': 'IT', 'Price': 3999.0, 'Volume': 0.75}
{'Name': ' АЛЬТЕ РОККЕ БАРОЛ КРСХ', 'Color': 'Красное', 'Acidity': 'Сухое', 'Country': 'IT', 'Price': 1499.004, 'Volume': 0.75}